In [5]:
import mysql.connector
 
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Cruchu19"   
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")

# mycursor = mydb.cursor()
# query = ('DROP DATABASE medpractice')
# mycursor.execute(query)


CONNECTION SUCCESSFUL


In [6]:
#create a database
mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE medpractice")

In [7]:
#create tables

#patients
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.patients (SSN VARCHAR(11) PRIMARY KEY, \
                  name VARCHAR(255) NOT NULL, address VARCHAR(255) NOT NULL, birth_date DATETIME NOT NULL, \
                  physician_id VARCHAR(11) NOT NULL)")

#physicians
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.physicians (SSN VARCHAR(11) PRIMARY KEY, \
                  name VARCHAR(255) NOT NULL, primary_specialty VARCHAR(255) NOT NULL, experience_years TINYINT NOT NULL, \
                  CHECK (experience_years >= 0))")

#pharmacies
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.pharmacies (id VARCHAR(255) PRIMARY KEY, \
                  name VARCHAR(255) NOT NULL, address VARCHAR(255) NOT NULL, phone VARCHAR(50) NOT NULL)")

#drugs
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.drugs (id VARCHAR(255) NOT NULL UNIQUE, \
                  name VARCHAR(255) PRIMARY KEY)")
#prescriptions
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.prescriptions (id VARCHAR(255) PRIMARY KEY, \
                  patient_id VARCHAR(11) NOT NULL, physician_id VARCHAR(11) NOT NULL, drug_name VARCHAR(255) NOT NULL, \
                  date DATE NOT NULL, quantity SMALLINT NOT NULL, CHECK (quantity > 0))")

#adverse_interactions
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.adverse_interactions (drug_name1 VARCHAR(255), \
                  drug_name2 VARCHAR(255), PRIMARY KEY(drug_name1,drug_name2))")

#alerts 
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.alerts (patient_id VARCHAR(11) NOT NULL, \
                  physician_id VARCHAR(11) NOT NULL, alert_date DATE NOT NULL, drug1 VARCHAR(255) NOT NULL, drug2 VARCHAR(255) NOT NULL)")

#pharmacy_fills
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.pharmacy_fills (pharmacy_id VARCHAR(255),prescription_id VARCHAR(255), \
                   date DATE NOT NULL, cost DECIMAL(6,2) NOT NULL, \
                  CHECK (cost >= 0), PRIMARY KEY(prescription_id,pharmacy_id))")

#companies 
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.companies (id VARCHAR(255) PRIMARY KEY, \
                  name VARCHAR(255) NOT NULL, address VARCHAR(255) NOT NULL, contact_phone VARCHAR(50) NOT NULL, \
                  contact_name VARCHAR(255) NOT NULL)")

#contracts
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE medpractice.contracts (id VARCHAR(255) PRIMARY KEY, \
                  drug_name VARCHAR(255) NOT NULL, dosage SMALLINT NOT NULL,pharmacy_id VARCHAR(255) NOT NULL,company_id VARCHAR(255) NOT NULL,   \
                  CHECK (dosage > 0),quantity INT NOT NULL, CHECK (quantity > 0),date DATE NOT NULL, price DECIMAL(6,2) NOT NULL, CHECK (price >= 0))")

In [8]:
#create foreign keys

#patients
query = ('ALTER TABLE medpractice.patients ADD foreign key(physician_id) \
          references medpractice.physicians(ssn)')
mycursor.execute(query)

#prescriptions
query = ('ALTER TABLE medpractice.prescriptions ADD foreign key(patient_id) \
          references medpractice.patients(ssn)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.prescriptions ADD foreign key(physician_id) \
          references medpractice.physicians(ssn)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.prescriptions ADD foreign key(drug_name) \
          references medpractice.drugs(name)')
mycursor.execute(query)

#adverse_interactions
query = ('ALTER TABLE medpractice.adverse_interactions ADD foreign key(drug_name1) \
          references medpractice.drugs(name)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.adverse_interactions ADD foreign key(drug_name2) \
          references medpractice.drugs(name)')
mycursor.execute(query)

#alerts
query = ('ALTER TABLE medpractice.alerts ADD foreign key(patient_id) \
          references medpractice.patients(ssn)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.alerts ADD foreign key(physician_id) \
          references medpractice.physicians(ssn)')
mycursor.execute(query)
query = ('USE medpractice')
mycursor.execute(query)
query = ('CREATE INDEX idx_prescriptions_patient_drug ON prescriptions (patient_id, drug_name)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.alerts ADD foreign key(patient_id,drug1) \
          references medpractice.prescriptions(patient_id,drug_name)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.alerts ADD foreign key(patient_id,drug2) \
          references medpractice.prescriptions(patient_id,drug_name)')
mycursor.execute(query)

#pharmacy_fills
query = ('ALTER TABLE medpractice.pharmacy_fills ADD foreign key(prescription_id) \
          references medpractice.prescriptions(id)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.pharmacy_fills ADD foreign key(pharmacy_id) \
          references medpractice.pharmacies(id)')
mycursor.execute(query)

#contracts
query = ('ALTER TABLE medpractice.contracts ADD foreign key(company_id) \
          references medpractice.companies(id)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.contracts ADD foreign key(pharmacy_id) \
          references medpractice.pharmacies(id)')
mycursor.execute(query)
query = ('ALTER TABLE medpractice.contracts ADD foreign key(drug_name) \
          references medpractice.drugs(name)')
mycursor.execute(query)

In [9]:
#connect to a specific database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Cruchu19", 
  database = "medpractice" 
)
mycursor = mydb.cursor()


In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("/Users/hudasaeed/Downloads/physicians.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [12]:
data = pd.read_csv("/Users/hudasaeed/Downloads/patients.csv")
data['birthdate'] = [s.replace('T', ' ').replace('Z', '') for s in data['birthdate']]

for i,row in data.iterrows():
            sql = "INSERT INTO patients VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [13]:
data = pd.read_csv("/Users/hudasaeed/Downloads/pharmacies.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [14]:
data = pd.read_csv("/Users/hudasaeed/Downloads/drugs.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [16]:
import datetime

data = pd.read_csv("/Users/hudasaeed/Downloads/prescriptions.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO prescriptions VALUES (%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [17]:
data = pd.read_csv("/Users/hudasaeed/Downloads/adverse_reactions.csv")
for i,row in data.iterrows():
            sql = "INSERT INTO adverse_interactions VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [18]:
data = pd.read_csv("/Users/hudasaeed/Downloads/alerts.csv")
data['alert_date'] = pd.to_datetime(data['alert_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO alerts VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [19]:
data = pd.read_csv("/Users/hudasaeed/Downloads/pharmacy_fills.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO pharmacy_fills VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [20]:
data = pd.read_csv("/Users/hudasaeed/Downloads/companies.csv")

for i,row in data.iterrows():
            sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [21]:
data = pd.read_csv("/Users/hudasaeed/Downloads/contracts.csv")
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

for i,row in data.iterrows():
            sql = "INSERT INTO contracts VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not autocommitted by default, so we 
            # must commit to save our changes
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
